# WORD2VEC

## Working with Word2Vec with Gensim


### Objectives
- Visualize word vectorization
- Prepare gensim environment
- Fit word2vec models
- Interpret latent variables/vectors
- Find similar words and word pairs
- Use externally-trained matrices of latent features

We have been working with a number of techniques and tools that help us navigate the world of NLP. For example, we have CountVectorizer:

In [ ]:
%matplotlib inline
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns  
import numpy as np
import matplotlib.pyplot as plt

text = ['That is should come to this!', 'This above all: to thine own self be true.', 'Something is rotten in the state of Denmark.']
vectorizer = CountVectorizer(ngram_range=(1,2))

vectorizer.fit(text)
x = vectorizer.transform(text)
x_back = x.toarray()

pd.DataFrame(x_back, columns=vectorizer.get_feature_names())

>An Adverse Traits abou the Bag of Words model: Word Context & semantic meaning does not play a role.


>But then came Word2Vec!

>We will see that with Word2Vec, context does play a role and it can decipher relationships between words.

In [ ]:
words = ['king','queen', 'man', 'woman']  

vectors = np.array([[0.6,   0.8], 
                    [0.8, 0.6],
                   [.2,.4],[.4,.2]]
                 ) 

plt.plot(vectors[:,0], vectors[:,1], 'o')  
plt.xlim(0, 1.5)  
plt.ylim(0, 1.5)  
for word, x, y in zip(words, vectors[:,0], vectors[:,1]):  
    plt.annotate(word, (x, y), size=12)

More examples:
![](https://www.tensorflow.org/images/linear-relationships.png)

## First things first :

**1) Install Gensim: **

pip install gensim

**  2)  Make sure cython is installed ? **

cython -V

(if no cython):

pip install cython


** 3) test (Run the sample code in following cell) **


In [ ]:
import gensim
documents = ["Will this work?  I'm not sure.  If not go to step #4 (above)"]
texts = [[word for word in document.lower().split()]
         for document in documents]


print (texts)
model = gensim.models.Word2Vec(texts, size=100, window=5, min_count=1, workers=4,sg=1)


**4) If you see the following error : "UserWarning: C extension not loaded for Word2Vec"**


Do the following:

1.  pip uninstall gensim
2.  pip uninstall scipy 

3. pip install --no-cache-dir scipy==0.15.1
4. pip install --no-cache-dir gensim==0.12.1


**Refer to the following:** https://groups.google.com/forum/#!topic/gensim/isBqIhrw9mk

In [ ]:
# A 'Gensim' example: 

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

## Word2Vec Preprocessing

1. Tokenization   
2. Remove stop words    
3. Convert to lowercase     
4. Others: stemming.. 

In [ ]:
# The type of input that Word2Vec is looking for.. 
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

print (texts)

###  2: Word Representation

Learn a continuous representation of words.
Each word (w) is associated with it's own word vector

In [ ]:
import gensim  # using skip-gram
model = gensim.models.Word2Vec(texts, size=100, window=5, min_count=1, workers=2, sg=1)

[See Wiki on arguments](https://en.wikipedia.org/wiki/Word2vec)

A Word2vec model can be trained with hierarchical softmax and/or negative sampling. To approximate the conditional log-likelihood a model seeks to maximize, the hierarchical softmax method uses a Huffman tree to reduce calculation. The negative sampling method, on the other hand, approaches the maximization problem by minimizing the log-likelihood of sampled negative instances. According to the authors, hierarchical softmax works better for infrequent words while negative sampling works better for frequent words and better with low dimensional vectors. As training epochs increase, hierarchical softmax stops being useful.

The size of the context window determines how many words before and after a given word would be included as context words of the given word. The recommended value is 10 for skip-gram and 5 for CBOW.

In [ ]:
# take a look at vocab
list(model.wv.vocab.items())[:7]

In [ ]:
#  and voila, we have our word vector 
print (model['trees'])

###  What do we have? Word Embeddings 

**A word embedding W : words → ℝn **

The output above is the result of 'word' projections in a latent space
of N dimensions, (N ~ size of NN layer we chose).     
Our float values above represent the coordinates for the word 'computer' in our 100-dimensional space!

Our high dimensional vectors stand in place for words.    
Note, that these dimensions are encoding 'latent' properties for 'computer' (such that 'queen' will be geometrically closer to 'king' than it would to be to (let's say) 'computer'. 

Word Embeddings are useful because:

1.  We can measure the semantic similarity between two words
2.  We can use these word vectors as features for various NLP supervised learning tasks (such as classification, sentiment analysis). 

### 3:  Skip-Gram Methods &  CBOW Methods : 

#### Skip-Gram example sentence:

>"We are on the cusp of deep learning for the masses"

For Context Window = 2:

*We could get the following training examples: (Where target word is __underlined__) *


__We__ are on 

We __are__ on the

We are __on__ the cusp


#### What's happening underneath the hood?:
We have input of skip-gram is a single word (Wi) **'learning'**, we will determine the probability of seeing the words (Wo) : 'of','deep', 'for','the'.

1. Transform our vobabulary into a 'bag of indices'

2. One-hot encode (input vectors) 

3. Randomly initialize the Weight Vectors

4. Get dot product: (Input vector.InputWeightMatrix) ~ (this is just the weight vector for 'learning')

5. Get dot product:  ('learning' weight vector).(Output Weight Matrix) 

6. Calculate Softmax probabilities:
    - What is the probability of 'seeing' the word 'deep' given that we've seen the word 'learning'?  -- >  Using SGD together with softmax regression, we will maximize the probability for 'deep' 
    - P(Wo|Wi) = (exp(Wi.Wo)/ sum(exp(Wi.Woj)   (sum~ sum of all Woj for all j in Vocabulary)

7. As always, we update our Weight matrix to reduce our errors:
    - Wi=Wi-a*ej*Wo

8. Repeat...


<img src='img/skip_gram.png'/>


### CBOW: 


CBOW: very similiar model with the inputs & outputs reversed.  The input layer consists of our word window 

<img src='img/CBOW.png'/>



### Let's work with a corpus from NLTK

In [ ]:
# WARNING: Use Scroll bar with NLTK download... or else?
import nltk
#nltk.download()

In [ ]:
nltk.download("gutenberg")

In [ ]:
# Build corpus from 
import os
from nltk.corpus import stopwords

# stop = stopwords.words('english')
stop = ['?','!','.',',',':',';']

#creating our iterator


# An Illustration.. 

import os

class MySentences(object):
    # a memory-friendly way to load a large corpora
     def __init__(self, dirname):
            self.dirname = dirname
 
     def __iter__(self):
        # iterate through all file names in our directory
         for fname in os.listdir(self.dirname):
                for line in open(os.path.join(self.dirname, fname), encoding="ISO-8859-1"):
                    word=line.lower().split()
                    if word not in stop:
                        yield word

sentences = MySentences('/home/ubuntu/nltk_data/corpora/gutenberg') 
model = gensim.models.Word2Vec(sentences,size=100,min_count=3,workers=4)

In [ ]:
model.most_similar('the' ,topn=8)

In [ ]:
# Similarity

model.similarity('man','flower')

In [ ]:
# Compute cosine_similarity

model.n_similarity(['dog', 'houses'], ['cat', 'cats'])

In [ ]:
model.doesnt_match("rabbit cow raven turtle".split())

In [ ]:
model.wv.evaluate_word_pairs(os.path.join(module_path, 'test_data','wordsim353.tsv'))


### If you want to refine your model, you'll need more data:
#### Let's work with Google's pre-trained [Word2Vect Model](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)

In [ ]:
import os

# Setup nltk corpora path and Google Word2Vec location
google_vec_file = '/Users/jb/Downloads/GoogleNews-vectors-negative300.bin'

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [ ]:
model.most_similar('king' ,topn=4)

In [ ]:
model.most_similar('president' ,topn=4)

In [ ]:
model.n_similarity(['king', 'man'], ['queen', 'woman'])

Other very cool methods!:      


Doc2Vec extends the word2vec algorithm to larger blocks of texts (paragraphs, documents, articles):    
- https://radimrehurek.com/gensim/models/doc2vec.html
- http://learningaboutdata.blogspot.com/2014/06/plotting-word-embedding-using-tsne-with.html
- https://districtdatalabs.silvrback.com/modern-methods-for-sentiment-analysis
- https://radimrehurek.com/gensim/models/word2vec.html
- https://www.tensorflow.org/tutorials/word2vec

###   Some things to keep in Mind when using Word2Vec:

Word2vec requires a lot of data to train.

As we've illustrated, you can download pretrained vectors. However, if you would need to train your own data 
you will need a lot of it!  (Think Millions of words) 
